In [1]:
import A_star as As

In [2]:
arad = As.No('Arad', 0, None, None)

In [3]:
p_teste = As.Problema(As.estados_romenia, arad, lambda no: no.estado == 'Bucharest')
b_teste = As.A_Star(p_teste)

In [4]:
b_teste.executar()

AttributeError: 'NoneType' object has no attribute 'funcao_avaliacao'

In [ ]:
npo = As.No('Sibiu', 113, None, None)

In [ ]:
print(arad)
print(npo)

(Arad, c = 0, d = 366)
(Sibiu, c = 113, d = 366)


In [ ]:
(arad == npo)

False

In [ ]:
366-253

113